# SWIS-100
## Set up run (interactive/single-shot)

In [ ]:
import os
import numpy as np
import pandas as pd
import swis100 as swis


In [ ]:
display(swis.solar_pu_raw)
display(swis.wind_pu_raw)
display(swis.elec_load_data_raw)
display(swis.assumptions_raw)

In [ ]:
run_config = {}

run_config['use_pyomo'] = False
run_config['solver_name'] = 'cbc'

run_config['assumptions_year'] = 2030 # Used to select projected nominal technology costs

# https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/eurofxref-graph-usd.en.html
# Change from 7 July 2019 to 8 July 2020
# Minimum (20 March 2020): 1.0707 - Maximum (9 March 2020): 1.1456 - Average: 1.1058
run_config['usd_to_eur'] = (1.0/1.1058)

run_config['Nyears'] = 2
run_config['snapshot_interval'] = 12 # hours
run_config['load_year_start'] = 2014

# Available year(s) for variable electricity load data: 2014-2019 inclusive
# Available year(s) for transport load data: 1991-2017 inclusive 

# If want test on constant electricity load, set this True
run_config['constant_elec_load_flag'] = False
if run_config['constant_elec_load_flag'] :
    run_config['constant_elec_load (GW)'] = 1.0
else :
    run_config['elec_load_scope']="IE+NI"

# If want test on constant surface transport load, set this True
run_config['constant_surface_transport_load_flag'] = False
if run_config['constant_surface_transport_load_flag'] :
    run_config['constant_surface_transport_load (GW)'] = 1.0

run_config['nuclear_SMR_min_p (GW)'] = 0.0
run_config['nuclear_SMR_max_p (GW)'] = +np.inf

# NB: Hardwired geo-location for renewables_ninja "IE"
# Available year(s) for weather data: solar 1985-2015 inclusive, wind 1980-2016
run_config['weather_year_start'] = 2010

# Set nominal VRE marginal costs to contol dispatch priority. Lower cost gets higher
# priority). Set all to non-zero so that curtailment/dispatch down at source is 
# preferred over discarding energy from storage (though presumably as long as there are
# *any* fixed costs of storage, this would be avoided by minimising storage size anyway).
run_config['solar_marginal_cost'] =0.03 # €/MWh
run_config['onshore_wind_marginal_cost'] =0.02 # €/MWh
run_config['offshore_wind_marginal_cost'] =0.01 # €/MWh

run_config['offshore_wind_min_p (GW)'] = 0.0
run_config['offshore_wind_max_p (GW)'] = +np.inf
#run_config['offshore_wind_max_p (GW)'] = 20.0

run_config['onshore_wind_min_p (GW)'] = 0.0 # Config var does still need to be defined...
#run_config['onshore_wind_min_p (GW)'] = 4.2548+1.2763
    # Sep 2020 installed capacity (all onshore)
    # IE: 4.2548, NI: 1.2763
    # per eirgrid "System and Renewable Data Summary Report"
    # http://www.eirgridgroup.com/site-files/library/EirGrid/System-and-Renewable-Data-Summary-Report.xlsx
run_config['onshore_wind_max_p (GW)'] = +np.inf
#run_config['onshore_wind_max_p (GW)'] = 8.2 
    # 2030 ambition in CAP-2019 (RES-E 70%) - not necessarily upper limit

run_config['solar_min_p (GW)'] = 0.0
run_config['solar_max_p (GW)'] = +np.inf
#run_config['solar_max_p (GW)'] = 1.5 
    # 2030 ambition in CAP-2019, via NDP (RES-E 55%) - not necessarily upper limit

#run_config['IC_min_p (GW)'] = 0.5+0.5 # EWIC + Moyle
run_config['IC_min_p (GW)'] = 0.0 # Config var does still need to be defined...
#run_config['IC_max_p (GW)'] = +np.inf # Unlimited IC
#run_config['IC_max_p (GW)'] = 0.5+0.5+0.8 # EWIC + Moyle + Celtic
#run_config['IC_max_p (GW)'] = 0.5+0.5 # EWIC + Moyle
run_config['IC_max_p (GW)'] = 0.0 # No IC

run_config['IC_max_e (TWh)'] = +np.inf

run_config['Battery_max_p (MW)'] = +np.inf
run_config['Battery_max_e (MWh)'] = +np.inf

run_config['H2_electrolysis_tech'] = 'default'
#run_config['H2_electrolysis_tech'] = 'Nicola-NEL'

run_config['H2_electrolysis_max_p (GW)'] = +np.inf
run_config['H2_CCGT_max_p (GW)'] = +np.inf
run_config['H2_OCGT_max_p (GW)'] = +np.inf

run_config['H2_storage_tech'] = 'salt cavern'
#run_config['H2_storage_tech'] = 'rock cavern'
#run_config['H2_storage_tech'] = 'steel tank'

run_config['H2_store_max_e (TWh)'] = +np.inf

run_config['BEV_min_p (GW)'] = 0.0
run_config['BEV_max_p (GW)'] = +np.inf
run_config['FCEV_min_p (GW)'] = 0.0
run_config['FCEV_max_p (GW)'] = +np.inf

display(run_config)
#print(run_config)

## Solve the system (interactive/single-shot)

In [ ]:
network = swis.solve_network(run_config)

In [ ]:
run_stats=swis.gather_run_stats(run_config, network)
with pd.option_context('display.max_rows', None): # Display ALL rows (no ellipsis)
    display(run_stats)

## Some one-off plotting

In [ ]:
Nyears = int(run_config['Nyears'])
weather_year_start = int(run_config['weather_year_start'])
weather_year_end = weather_year_start + (Nyears - 1)

plt_start = F"{weather_year_start}-01-01"
plt_stop = F"{weather_year_end}-12-31"

# NB: interactive plotting may be sloooow for large time windows!


In [ ]:
import matplotlib.pyplot as plt
# For non-interactive plots use this magic:
#%matplotlib inline

# For interactive plots use this magic (REQUIRES ipympl package installed!):
%matplotlib widget 

# Set plotsize in notebook
# https://www.mikulskibartosz.name/how-to-change-plot-size-in-jupyter-notebook/
plt.rcParams["figure.figsize"] = (8,4)

fig, ax = plt.subplots()

rename = {"ic-import" : "import",
          "ic-export" : "export",
          "nuclear-SMR" : "nuclear",
          "onshore wind" : "onshore wind",
          "offshore wind" : "offshore wind",
          "solar" : "utility solar PV",
          "battery discharge" : "battery discharge",
          "battery charge" : "battery charge",
          "H2 electrolysis" : "hydrogen electrolysis",
          "H2 CCGT" : "hydrogen CCGT",
          "H2 OCGT" : "hydrogen OCGT"}

rename = {""+k : v for k,v in rename.items()}

rename["local-elec-demand"] = "elec load"
rename["surface-transport-demand"] = "surface transport load"

# matplotlib named colors:
# https://matplotlib.org/3.1.0/gallery/color/named_colors.html
colors = {"import" : "green",
           "export" : "green",
           "nuclear" : "aqua",
           "onshore wind" : "royalblue",
           "offshore wind" : "cornflowerblue",
           "utility solar PV" : "yellow",
           "battery discharge" : "gray",
           "battery charge" : "gray",
           "elec load" : "black",
           "surface transport load" : "rosybrown",
           "hydrogen electrolysis" : "magenta",
           "hydrogen CCGT" : "red",
           "hydrogen OCGT" : "pink"
          }

# KLUDGE: for some reason (solver tolerance?) some solution values that should be strictly 
# positive or negative may be infinitesimally (< 10e-10) of the other sign. This will cause 
# df.plot(stacked=True) to throw an error (requires all values to have same sign, positive or 
# negative). So we do a somewhat ugly ".round(10)" kludge to fix it...

positive = pd.concat((network.generators_t.p,
                      -network.links_t.p1[["ic-import","H2 CCGT","H2 OCGT","battery discharge"]]),
                     axis=1).rename(columns=rename).round(10)
negative = pd.concat((-network.loads_t.p,
                      -network.links_t.p0[["ic-export","H2 electrolysis","battery charge"]]),
                     axis=1).rename(columns=rename).round(10)

print((abs(positive.sum(axis=1) + negative.sum(axis=1)) > 0.1).any())

load_max=network.loads_t.p.sum(axis=1).max()
gen_max=positive.sum(axis=1).max()
demand_max=negative.sum(axis=1).min()

print([load_max,demand_max,gen_max])

# Set y_lim top extra large (*2.5) to make space for the legend...
positive.loc[plt_start:plt_stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                            ylim=(1.2*demand_max, 2.5*gen_max),
                            color=[colors[i] for i in positive.columns])

negative.loc[plt_start:plt_stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                           ylim=(1.2*demand_max, 2.5*gen_max),
                            color=[colors[i] for i in negative.columns])

#ax.set_ylim(1.2*demand_max, 2.0*gen_max)
#ax.set_xlim([plt_start,plt_stop])
ax.set_ylabel("Dispatch (generation is +ve, demand is -ve) [MW]")
ax.legend(ncol=3,loc="upper left")

#fig.tight_layout()
#fig.savefig("img/{}-{}-{}-{}.png".format(ct,scenario,start,stop),dpi=100)

In [ ]:
import matplotlib.pyplot as plt
# For non-interactive plots use this magic:
# %matplotlib inline

# For interactive plots use this magic (REQUIRES ipympl package installed!):
%matplotlib widget 

plt.rcParams["figure.figsize"] = (8,4)

fig, ax = plt.subplots()

"H2 store underground"
"H2 store tank"
"battery storage"

rename = {"remote-elec-grid-buffer" : "IC buffer", "H2 store" : "H2 store",
    "battery storage" : "battery"}

rename = {""+k : v for k,v in rename.items()}

colors = {"IC buffer" : "m",
        "H2 store" : "b",
        "battery" : "gray"}

storage = network.stores_t.e[["remote-elec-grid-buffer","H2 store",
                              "battery storage"]].rename(columns=rename).round(10)
display(storage)

storage.loc[plt_start:plt_stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                              color=[colors[i] for i in storage.columns])

e_max = storage.sum(axis=1).max()
e_min = storage.sum(axis=1).min()

#ax.set_ylim([demand_max,gen_max])
print([demand_max,gen_max])
ax.set_ylim([0.0, 1.2*e_max])
ax.set_xlim([plt_start,plt_stop])
ax.set_ylabel("Storage [MWh]")
ax.legend(ncol=3,loc="upper left")

fig.tight_layout()

#fig.savefig("img/{}-{}-{}-{}.png".format(ct,scenario,start,stop),dpi=100)

## Dump (volatile) run data to (persistent) file storage (just in case...)

**FIXME:** This is very rough and ready - needs much better approach!
Currently offers no ready way to re-load/re-run previous configs... :-(


In [ ]:
run_id = 'scratch'
run_dir='runs/single-run/'+run_id
os.makedirs(run_dir,exist_ok=True)

#network.export_to_netcdf(run_dir+'/network.nc') # Uncomment to save all network object data
run_config_series=pd.Series(run_config, dtype=object, name=run_id)
run_config_df=run_config_series.to_frame().transpose()
run_config_df.to_excel(run_dir+'/run_config.ods')
run_stats.to_excel(run_dir+'/run_stats.ods')